<a href="https://colab.research.google.com/github/DavideRutigliano/frigate-models/blob/main/notebooks/YOLO_NAS_Pretrained_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
%%capture
!pip install super_gradients
!git clone https://github.com/Deci-AI/super-gradients.git /content/super_gradients_folder
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/pretrained_models.py
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/utils/checkpoint_utils.py
!pip install -e /content/super_gradients_folder
!pip install onnxruntime

## Select model

In [2]:
model_name = 'yolonas_s' #@param ["yolonas_s", "yolonas_m", "yolonas_l"]
quantization = 'fp16' #@param ["fp16", "uint8"]
input_width = 320 #@param {type:"slider", min:320, max:640, step:320}
input_height = 320 #@param {type:"slider", min:320, max:640, step:320}

MODEL_FILENAME = f"{model_name}_{quantization}_{input_width}x{input_height}.onnx"

In [ ]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

MODEL_NAMES = {
    "yolonas_s": Models.YOLO_NAS_S,
    "yolonas_m": Models.YOLO_NAS_M,
    "yolonas_l": Models.YOLO_NAS_L,
}

model = models.get(MODEL_NAMES[model_name], pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, input_height, input_width])

The console stream is logged into /root/sg_logs/console.log


[2025-03-14 08:27:56] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


In [ ]:
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.conversion.conversion_enums import ExportQuantizationMode

quantization_mode = ExportQuantizationMode.FP16 if quantization == "fp16" else ExportQuantizationMode.INT8
model.export(
  MODEL_FILENAME,
  input_image_shape=(input_height, input_width),
  num_pre_nms_predictions=300,
  max_predictions_per_image=5,
  nms_threshold=0.7,
  confidence_threshold=0.4,
  output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
  quantization_mode=quantization_mode
)

In [ ]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.uint8)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"input": dummy_input})

In [ ]:
from google.colab import files

files.download(MODEL_FILENAME)